In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Parse backtest result

In [10]:
import json

file_name = '../backtest_results/backtest-20220416-041017.json'

with open(file_name) as f:
    json_file = json.load(f)

In [11]:
start_balance = json_file['balance'][0]
start_balance

{'date': '2021-08-07 17:00:00',
 'ATOM': 0.18,
 'MATIC': 0.0,
 'ETH': 0.0171,
 'BTC': 0.00143,
 'BNB': 0.09727621,
 'LINK': 0.0,
 'DOT': 1.69,
 'XRP': 0.0,
 'LTC': 0.0,
 'UNI': 0.0,
 'AVAX': 0.16,
 'ADA': 0.0,
 'SOL': 0.71,
 'LUNA': 0.57,
 'DOGE': 0.0}

In [12]:
final_balance = json_file['balance'][-1]
final_balance

{'date': '2022-04-14 17:00:00',
 'ATOM': 0.18,
 'MATIC': 1.8778999999999986,
 'ETH': 0.0136961,
 'BTC': 0.00143,
 'BNB': 0.09727621,
 'LINK': 0.07952999999999993,
 'DOT': 1.06956,
 'XRP': 0.0,
 'LTC': 0.0,
 'UNI': 1.0474899999999998,
 'AVAX': 0.16,
 'ADA': 0.0,
 'SOL': 0.47,
 'LUNA': 0.82798,
 'DOGE': 14.906999999999996,
 'USDT': 0.09820596000000492}

In [13]:
from prettytable import *
msg = PrettyTable()
msg.field_names = ["Coin", "From", "To", "Growth"]


for key in final_balance:
    end = final_balance[key]
    start = start_balance[key] if key in start_balance else '0.0'
    growth = ''
    if key != 'date':
        s = float(start)
        e = float(end)
        if s > 0:
            growth = round(((e - s)/s)*100,2)
        else:
            growth = 'inf'
    msg.add_row([
        key,
        start,
        end,
        growth
    ])

msg.align = "l"
print(msg)

+-------+---------------------+---------------------+--------+
| Coin  | From                | To                  | Growth |
+-------+---------------------+---------------------+--------+
| date  | 2021-08-07 17:00:00 | 2022-04-14 17:00:00 |        |
| ATOM  | 0.18                | 0.18                | 0.0    |
| MATIC | 0.0                 | 1.8778999999999986  | inf    |
| ETH   | 0.0171              | 0.0136961           | -19.91 |
| BTC   | 0.00143             | 0.00143             | 0.0    |
| BNB   | 0.09727621          | 0.09727621          | 0.0    |
| LINK  | 0.0                 | 0.07952999999999993 | inf    |
| DOT   | 1.69                | 1.06956             | -36.71 |
| XRP   | 0.0                 | 0.0                 | inf    |
| LTC   | 0.0                 | 0.0                 | inf    |
| UNI   | 0.0                 | 1.0474899999999998  | inf    |
| AVAX  | 0.16                | 0.16                | 0.0    |
| ADA   | 0.0                 | 0.0                 | i

In [14]:
from sqlitedict import SqliteDict
from dateutil import parser
from datetime import timedelta

sqlite_cache = SqliteDict("../data/backtest_cache.db")

values = {}
start_date = parser.parse(start_balance['date'])
starting_value_in_usdt = 0
for k in start_balance:
    if k != 'date':
        v = float(start_balance[k])
        if k == 'USDT':
            starting_value_in_usdt += v
        else:
            target_date = start_date.strftime("%d %b %Y %H:%M:%S")
            key = f"{k}USDT - {target_date}"
            usdt_val = sqlite_cache.get(key, None)
            starting_value_in_usdt += usdt_val * v
            values[k] = {
                'start': v,
                'start_price': usdt_val,
                'start_value': usdt_val * v
            }

final_date = parser.parse(final_balance['date'])
final_date = final_date - timedelta(days=1)
finaling_value_in_usdt = 0
for k in final_balance:
    if k != 'date':
        v = float(final_balance[k])
        if k == 'USDT':
            finaling_value_in_usdt += v
        else:
            target_date = final_date.strftime("%d %b %Y %H:%M:%S")
            key = f"{k}USDT - {target_date}"
            usdt_val = sqlite_cache.get(key, None)
            #print(f'{key} :\tusdt_val * v = {usdt_val * v}')
            finaling_value_in_usdt += usdt_val * v
            values[k] = {
                **values[k],
                **{
                    'final': v,
                    'final_price': usdt_val,
                    'final_value': usdt_val * v
                }
            }
print(f'{starting_value_in_usdt} USDT -> {finaling_value_in_usdt} USDT')

msg = PrettyTable()
msg.field_names = ['coin', 'start_balance', 'final_balance', 'start_price', 'final_price', 'start_value', 'final_value']
for k in values:
    row = values[k]
    msg.add_row(
        [
            k,
            row['start'],
            row['final'],

            row['start_price'],
            row['final_price'],

            row['start_value'],
            row['final_value'],


        ]
    )
msg.align = "l"
print(msg)


224.84632572639998 USDT -> 315.119826859 USDT
+-------+---------------+---------------------+-------------+-------------+--------------------+--------------------+
| coin  | start_balance | final_balance       | start_price | final_price | start_value        | final_value        |
+-------+---------------+---------------------+-------------+-------------+--------------------+--------------------+
| ATOM  | 0.18          | 0.18                | 13.835      | 24.89       | 2.4903             | 4.4802             |
| MATIC | 0.0           | 1.8778999999999986  | 1.15529     | 1.414       | 0.0                | 2.655350599999998  |
| ETH   | 0.0171        | 0.0136961           | 3090.24     | 3086.5      | 52.843104          | 42.27301265        |
| BTC   | 0.00143       | 0.00143             | 43705.0     | 41005.59    | 62.49815           | 58.637993699999996 |
| BNB   | 0.09727621    | 0.09727621          | 351.84      | 416.9       | 34.2256617264      | 40.554451949       |
| LINK  | 

No coin gain
224.84632572639998 USDT -> 178.37692496499997 USDT
+-------+---------------+-----------------------+-------------+-------------+--------------------+----------------------+
| coin  | start_balance | final_balance         | start_price | final_price | start_value        | final_value          |
+-------+---------------+-----------------------+-------------+-------------+--------------------+----------------------+
| ATOM  | 0.18          | 0.18                  | 13.835      | 24.89       | 2.4903             | 4.4802               |
| MATIC | 0.0           | 0.07239999999998759   | 1.15529     | 1.414       | 0.0                | 0.10237359999998244  |
| ETH   | 0.0171        | 7.530000000000037e-05 | 3090.24     | 3086.5      | 52.843104          | 0.23241345000000113  |
| BTC   | 0.00143       | 0.00143               | 43705.0     | 41005.59    | 62.49815           | 58.637993699999996   |
| BNB   | 0.09727621    | 0.09727621            | 351.84      | 416.9       | 34.2256617264      | 40.554451949         |
| LINK  | 0.0           | 0.007039999999999935  | 24.344      | 14.06       | 0.0                | 0.0989823999999991   |
| DOT   | 1.69          | 0.008680000000000021  | 20.69       | 18.1        | 34.966100000000004 | 0.1571080000000004   |
| XRP   | 0.0           | 0.0                   | 0.8027      | 0.718       | 0.0                | 0.0                  |
| LTC   | 0.0           | 0.0                   | 154.66      | 109.6       | 0.0                | 0.0                  |
| UNI   | 0.0           | 6.366709999999999     | 28.147      | 9.6         | 0.0                | 61.12041599999999    |
| AVAX  | 0.16          | 0.16                  | 15.012      | 79.59       | 2.40192            | 12.7344              |
| ADA   | 0.0           | 0.0                   | 1.4511      | 0.961       | 0.0                | 0.0                  |
| SOL   | 0.71          | 0.0                   | 38.358      | 104.6       | 27.234179999999995 | 0.0                  |
| LUNA  | 0.57          | 0.0004499999999998394 | 14.363      | 86.85       | 8.18691            | 0.03908249999998605  |
| DOGE  | 0.0           | 0.44399999999996      | 0.24042     | 0.1396      | 0.0                | 0.061982399999994414 |
+-------+---------------+-----------------------+-------------+-------------+--------------------+----------------------+